In [ ]:
import numpy as np
import cvxopt as cvx
import control as con
from numpy import linalg as LA

In [ ]:
from cvxpy import *

# Create two scalar optimization variables.
x = Variable()
y = Variable()

# Create two constraints.
constraints = [x + y == 1,
               x - y >= 1]

# Form objective.
obj = Minimize(square(x - y))

# Form and solve problem.
prob = Problem(obj, constraints)
prob.solve()  # Returns the optimal value.
print "status:", prob.status
print "optimal value", prob.value
print "optimal var", x.value, y.value

In [ ]:
print installed_solvers()

In [ ]:
### Seite 37 (im Text zwischen (4.12) und (4.13))
def _N(n):
    return np.diag([p for p in xrange(-n, 0, 1)])

### Seite 55; (4.64)
def _M(n):
    return np.diag([p for p in xrange(0, n, 1)])

### Seite 55; (4.64)
def _P(l, k, n):
    I = np.eye(n)
    Mn = _M(n)
    P = I
    if k == 0:
        pass # P=I
    else:
        for q in xrange(0, k, 1):
            P = P * ((l-q)*I + Mn)
    return cvx.matrix(P)

In [ ]:
###################################################
# Boris Diss Hydraulisches Positioniersystem S.61 #
###################################################
A = np.matrix([[0,      1,        0],
               [0, -0.126,     83.5],
               [0,  -6.89, -0.00707]])
 
b = np.matrix([[0],[0],[-55.51]])
c = np.matrix([1, 0, 0])

print "Poles:\n", np.matrix(con.matlab.pole(con.matlab.ss(A, b, c, 0))).T

u_max = 1
n = len(b)

X0 = [np.matrix([-13.38, -10.7, -2.58]).T,
      np.matrix([-13.38, -10.7,  2.58]).T,
      np.matrix([-13.38,  10.7, -2.58]).T,
      np.matrix([-13.38,  10.7,  2.58]).T,
      np.matrix([ 13.38, -10.7, -2.58]).T,
      np.matrix([ 13.38, -10.7,  2.58]).T,
      np.matrix([ 13.38,  10.7, -2.58]).T,
      np.matrix([ 13.38,  10.7,  2.58]).T]

x_max = [13.38, 10.7, 2.58]

#print "A:\n", A
#print "a:\n", a
#print "b:\n", b
#print "c:\n", c
p_min = 0.2
mu_star = 1.5
zeta = 2.5
zeta_star = 1.5

k0 = 1e-3*np.matrix([-24.23, -0.2122, -10.5]).T
k1 = 1e-3*np.matrix([-40.13, -0.1396, -15.10]).T
k0_star = 1e-3*np.matrix([-11.96, -0.1111, -5.18]).T
k1_star = 1e-3*np.matrix([-83.35, -0.0477, -33.86]).T

##### Entwurf parameter #####
beta=2 # beta >=1 !

In [ ]:
import cvxpy

R0 = cvxpy.Semidef(n)
R1 = cvxpy.Semidef(n)

G = cvxpy.Semidef(n)
G_star = cvxpy.Semidef(n)

D = cvxpy.Semidef(n)
D_star = cvxpy.Semidef(n)

# Parameter
alpha = 1-p_min
beta = 1+p_min
    
A0 = A-b*k0.T
A1 = A-b*k1.T

S0 = A1.T*R1+R1*A1
S1 = A0.T*R1+R1*A0 + A1.T*R0+R0*A1
S2 = A0.T*R0+R0*A0

A0_star = A-b*k0_star.T
A1_star = A-b*k1_star.T

S0 = A1.T*R1+R1*A1
S1 = A0.T*R1+R1*A0 + A1.T*R0+R0*A1
S2 = A0.T*R0+R0*A0

S0_star = A1_star.T*R1+R1*A1_star
S1_star = A0_star.T*R1+R1*A0_star + A1_star.T*R0+R0*A1_star
S2_star = A0_star.T*R0+R0*A0_star

# Create two constraints.
constraints = [R0 >> 0,
               R0 + R1 >> 0]

const4_39c = cvxpy.bmat([[-D , G],
                         [G.T, D]]) - \
             cvxpy.bmat([[S0 + 0.5*beta*S1 + 0.25*beta**2*S2, 0.25*alpha*(S1 + beta*S2)],
                         [0.25*alpha*(S1 + beta*S2),          0.25*alpha**2*S2]]) >> 0

const4_39d = [cvxpy.bmat([[1, X0[i].T],
                          [X0[i], R0+R1]]) >> 0
                              for i in range(0, len(X0))]

const4_39e = cvxpy.bmat([[1,    k0.T],
                         [k0,   R0]]) + \
             1/p_min * cvxpy.bmat([[0,  k1.T],
                                  [k1, R1]]) >> 0

const4_39f = cvxpy.bmat([[1,    k0.T],
                         [k0,   R0]]) + \
             cvxpy.bmat([[0,  k1.T],
                         [k1, R1]]) >> 0

const4_39g = cvxpy.bmat([[-D_star , G_star],
                         [G_star.T, D_star]]) - \
             cvxpy.bmat([[S0_star + 0.5*beta*S1_star + 0.25*beta**2*S2_star, 0.25*alpha*(S1_star + beta*S2_star)],
                         [0.25*alpha*(S1_star + beta*S2_star),          0.25*alpha**2*S2_star]]) >> 0

constraints.append(const4_39c)
constraints.extend(const4_39d)
constraints.append(const4_39e)
constraints.append(const4_39f)
constraints.append(const4_39g)

# Form objective.
obj = cvxpy.Minimize(cvxpy.trace(R0+1/p_min*R1))

# Form and solve problem.
prob = cvxpy.Problem(obj, constraints)
prob.solve(solver=cvxpy.SCS)  # Returns the optimal value.
print "status:", prob.status
print "optimal value", prob.value
print "R0", R0.value
LA.cholesky(R0.value)
print "R1", R1.value
LA.cholesky(R1.value)
print "ok"